In [1]:

import ipywidgets as widgets
from IPython.display import display
jupyter_config = {
    "load_extensions": {
        "widgetsnbextension": True
    }
}


import os, re, csv, copy, json, warnings
from itertools import cycle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
from matplotlib.colors import Normalize
from matplotlib.lines import Line2D
from matplotlib.ticker import FormatStrFormatter
from IPython.display import display, HTML, clear_output
import _settings as settings
from io import BytesIO

#!jupyter nbextension enable --py widgetsnbextension
import ipywidgets as widgets
from ipywidgets import GridspecLayout, VBox, HBox, Layout, Output, HTML, interact, interact_manual
from my_functions_heatmapviz import (initialize_settings, proceed_with_label_specific_options, update_plot, update_filenames)#display_plotting_options, create_plotting_widgets, update_plot,update_filenames,
#    HeatmapDataHandler)


#global variables and assign the settings values to them
global valid_discrete_cmaps, default_hm_color, default_lp_color, default_avglp_color, default_filename_port, default_filename_land, images_folder_name
global active_vars, axis_number, num_sets, total_plots, style_map, cmap, max_sequence_length, chuck_size
global heatmap_legend_handles, lp_selected_color, avg_cmap
global selected_functions, selected_peptides, available_data_variables
global plot_heatmap, plot_zero, valid_discrete_cmaps, default_lp_color, default_avglp_color, default_filename_land


import _settings as settings

settings_dict = initialize_settings()
globals().update(settings_dict) 
# Import settings from _settings
spec_translate_list = settings.SPEC_TRANSLATE_LIST
valid_discrete_cmaps = settings.valid_discrete_cmaps
valid_gradient_cmaps = settings.valid_gradient_cmaps
default_hm_color = settings.default_hm_color
default_lp_color = settings.default_lp_color
default_avglp_color = settings.default_avglp_color
hm_selected_color = settings.hm_selected_color
cmap = settings.cmap
lp_selected_color = settings.lp_selected_color
avglp_selected_color = settings.avglp_selected_color
avg_cmap = settings.avg_cmap
legend_title = settings.legend_title

# Set base filename and proteins dictionary if needed
global base_filenamee
base_filename = 'heatmap_data_files/exported_heatmap_data'


In [21]:
class HeatmapDataHandler:
    def __init__(self):
        # Initialize all widgets here
        self.protein_dropdown = widgets.Dropdown(description='Protein:')
        self.grouping_variable_text = widgets.Text(description='Grouping Variable:')
        self.var_key_dropdown = widgets.Dropdown(description='Variable Key:')
        self.button_box = HBox([widgets.Button(description='Submit')])
        self.var_selection_output = widgets.Output()
        self.label_order_output = widgets.Output()
        self.available_data_variables = {}  # Populate this as needed
        self.label_widgets = {}  # Populate this as needed
        self.base_filename = base_filename

        # Initialize variables
        self.data_variables = self.extract_and_format_data()
                # Extract available protein IDs and names
        self.protein_mapping = {
            key.split('_')[0]: value['protein_name']
            for key, value in self.data_variables.items()
        }
        self.available_proteins = set([key.split('_')[0] for key in self.data_variables.keys()])
        self.available_grouping_vars = {
            protein: [key.split('_', 1)[1] for key in self.data_variables.keys() if key.startswith(protein)] for protein
            in self.available_proteins}
        self.selected_var_keys_list = []

        # Filtered Data Variables
        self.filtered_data_variables = {}
        self.available_data_variables = {}
        self.label_widgets = {}
        
        self.order_widgets = {}
        self.default_label_values = {}
        self.default_order_values = {}

        # Widgets
        self.create_widgets()

        # Additional attributes for plotting options
        self.ms_average_choice = None
        self.selected_peptides = []
        self.selected_functions = []
        self.legend_title = legend_title
        # Initialize variables
        self.bio_or_pep = 'no'  # Default value
        self.ms_average_choice = 'yes'  # Default value
        self.plot_heatmap = 'yes'  # Default value
        self.plot_zero = 'no'  # Default value

        self.user_protein_id = ''  # Will be set appropriately
        self.protein_name_short = ''  # Will be set appropriately

        self.label_order_output = widgets.Output()

    def load_complex_dict(self, base_path=None):
        if base_path is None:
            base_path = self.base_filename

        with open(os.path.join(base_path, 'metadata.json'), 'r') as f:
            metadata = json.load(f)

        result = {}
        for key, info in metadata.items():
            if info['type'] == 'nested':
                # Recursively load nested dictionaries
                result[key] = self.load_complex_dict(os.path.join(base_path, info['path']))
            elif info['type'] == 'dataframe':
                # Load DataFrame from CSV
                result[key] = pd.read_csv(os.path.join(base_path, info['filename']))
            elif info['type'] == 'direct':
                # Load direct value from metadata
                result[key] = info['value']

        return result

    # Function to extract and format data
    def extract_and_format_data(self):
        # Load the data from the saved directory
        loaded_data = self.load_complex_dict(self.base_filename)

        # Initialize the new dictionary
        data_variables = {}

        # Iterate over the loaded data to extract and reorganize it
        for protein_id, protein_data in loaded_data.items():
            protein_sequence = protein_data.get('protein_sequence')

            for grouping_var_name, group_info in protein_data.items():
                # Extract the required DataFrames and other information
                func_df = group_info.get('func_heatmap_df')
                abs_df = group_info.get('heatmap_df')
                label = grouping_var_name
                protein_sequence = group_info.get('protein_sequence')
                protein_name = group_info.get('protein_name')
                protein_species = group_info.get('protein_species')

                # Determine if the func_df is all None
                is_func_df_all_none = func_df.isnull().all().all() if func_df is not None else True

                # Create a unique key combining protein_id and grouping_var_name
                var_key = f"{protein_id}_{grouping_var_name}"

                # Populate the data_variables dictionary using the unique key
                data_variables[var_key] = {
                    'protein_id': protein_id,
                    'protein_sequence': protein_sequence,
                    'protein_name': protein_name,
                    'protein_species': protein_species,
                    'heatmap_df': abs_df,
                    'function_heatmap_df': func_df,
                    'label': label,
                    'is_func_df_all_none': is_func_df_all_none
                }

        return data_variables

    def chunk_dataframe(self, df, chunk_size, exclude_columns=3):

        # Select all rows and all but the last 'exclude_columns' columns
        df_subset = df.iloc[:, :-exclude_columns] if exclude_columns else df

        # Calculate the number of rows needed to make the last chunk exactly 'chunk_size'
        total_rows = df_subset.shape[0]
        remainder = total_rows % chunk_size
        if remainder != 0:
            # Rows needed to complete the last chunk
            rows_to_add = chunk_size - remainder

            # Create a DataFrame with zero values for the missing rows
            additional_rows = pd.DataFrame(np.zeros((rows_to_add, df_subset.shape[1])), columns=df_subset.columns)

            # Append these rows to df_subset
            df_subset = pd.concat([df_subset, additional_rows], ignore_index=True)

        # Create chunks of the DataFrame
        max_index = df_subset.index.max() + 1
        return [df_subset.iloc[i:i + chunk_size] for i in range(0, max_index, chunk_size)]

    # Function to process data variables
    def process_data_variables(self):
        chunk_size = 78
        # Print loaded dataframes and their labels
        for var, info in self.filtered_data_variables.items():
            if 'function_heatmap_df' in info:
                if info['is_func_df_all_none']:
                    display(HTML(f"<b>{var} - Label: {info['label']}</b>: Only absorbance data loaded."))
                else:
                    display(HTML(f"<b>{var} - Label: {info['label']}</b>: Absorbance and function data loaded."))

        # Dynamically generate the list of variable names based on loaded data
        variables = list(self.filtered_data_variables.keys())
        protein_id_list = []
        protein_name_list = []
        for var in variables:
            if var in self.filtered_data_variables and 'heatmap_df' in self.filtered_data_variables[var]:
                df = self.filtered_data_variables[var]['heatmap_df']
                df_func = self.filtered_data_variables[var]['function_heatmap_df']

                try:
                    self.filtered_data_variables[var]['peptide_counts'] = df['count']
                    self.filtered_data_variables[var]['ms_data'] = df['average']

                    self.filtered_data_variables[var]['max_peptide_counts'] = self.filtered_data_variables[var][
                        'peptide_counts'].max()
                    self.filtered_data_variables[var]['min_peptide_counts'] = self.filtered_data_variables[var][
                        'peptide_counts'].min()
                    self.filtered_data_variables[var]['max_ms_data'] = self.filtered_data_variables[var][
                        'ms_data'].max()
                    self.filtered_data_variables[var]['min_ms_data'] = self.filtered_data_variables[var]['ms_data'][
                        self.filtered_data_variables[var]['ms_data'] > 0].min()

                    self.filtered_data_variables[var]['amino_acids_chunks'] = [
                        self.filtered_data_variables[var]['protein_sequence'][i:i + chunk_size]
                        for i in range(0, len(self.filtered_data_variables[var]['protein_sequence']), chunk_size)
                    ]

                    self.filtered_data_variables[var]['peptide_counts_chunks'] = [
                        self.filtered_data_variables[var]['peptide_counts'][i:i + chunk_size]
                        for i in range(0, len(self.filtered_data_variables[var]['peptide_counts']), chunk_size)
                    ]

                    self.filtered_data_variables[var]['ms_data_chunks'] = [
                        self.filtered_data_variables[var]['ms_data'][i:i + chunk_size]
                        for i in range(0, len(self.filtered_data_variables[var]['ms_data']), chunk_size)
                    ]
                    self.filtered_data_variables[var]['ms_data_list'] = list(
                        self.filtered_data_variables[var]['ms_data'])
                    self.filtered_data_variables[var]['AA_list'] = df['AA'].tolist()

                    columns_to_include = df.columns.difference(['AA', 'COUNT'])
                    df_filtered = df[columns_to_include]

                    self.filtered_data_variables[var]['bioactive_peptide_abs_df'] = df_filtered
                    self.filtered_data_variables[var]['bioactive_peptide_chunks'] = self.chunk_dataframe(df_filtered,
                                                                                                         chunk_size=chunk_size)
                    self.filtered_data_variables[var]['bioactive_function_chunks'] = self.chunk_dataframe(df_func,
                                                                                                          chunk_size=chunk_size)
                    self.filtered_data_variables[var]['bioactive_peptide_func_df'] = df_func
                    protein_id_list.append(self.filtered_data_variables[var]['protein_id'])
                    protein_name_list.append(self.filtered_data_variables[var]['protein_name'])

                    print(f"All data structures for {var} have been created successfully.")

                except Exception as e:
                    display(HTML(f'<span style="color:red;">Error processing data for {var}: {e}</span>'))
            else:
                display(HTML(f'<span style="color:red;">{var} DataFrame is not loaded or does not exist.</span>'))

        user_protein_id_set = list(set(protein_id_list))
        user_protein_name_set = list(set(protein_name_list))

        if len(user_protein_id_set) > 1 and len(user_protein_name_set) == 1:
            self.user_protein_id = '_'.join(user_protein_id_set)
            self.protein_name_short = user_protein_name_set[0]

        elif len(user_protein_id_set) > 1 and len(user_protein_name_set) > 1:
            self.user_protein_id = '_'.join(user_protein_id_set)
            self.protein_name_short = '_'.join(user_protein_name_set)

        elif len(user_protein_name_set) == 1:
            self.user_protein_id = user_protein_id_set[0]
            self.protein_name_short = user_protein_name_set[0]

        self.available_data_variables = self.filtered_data_variables.copy()
    # Function to create order input widgets
    def create_order_input_widgets(self):
        description_layout_invisible = widgets.Layout(width='400px')

        self.label_widgets = {}
        self.order_widgets = {}
        for i, (var, info) in enumerate(self.available_data_variables.items()):
            self.label_widgets[var] = widgets.Text(
                value=info['label'],
                description='',
                layout=widgets.Layout(width='150px')
            )
            self.order_widgets[var] = widgets.IntText(
                value=i,
                description='',
                layout=description_layout_invisible,
            )
        # Optionally, you can return the widgets if needed
        # return self.label_widgets, self.order_widgets

    # Function to create widgets
    def create_widgets(self):
        # Create widgets for protein selection
        self.protein_dropdown = widgets.Dropdown(
            options=[("Select Protein", None)] + [
                (
                    f"{protein_id} - {self.protein_mapping.get(protein_id, 'Unknown')}",
                    protein_id
                )
                for protein_id in sorted(self.available_proteins)
            ],
            description='Protein ID:',
            layout=widgets.Layout(width='90%')
        )

        self.grouping_variable_text = widgets.Text(
            description='Search Term',
            layout=widgets.Layout(width='90%')
        )

        self.var_key_dropdown = widgets.SelectMultiple(
            description='Groups',
            disabled=False,
            layout=widgets.Layout(width='90%', height='100px')
        )

        self.add_group_button = widgets.Button(
            description='Add Group',
            button_style='success',
            layout=widgets.Layout(width='150px', height='30px')
        )
        self.search_button = widgets.Button(
            description='Search',
            button_style='info',
            layout=widgets.Layout(width='150px', height='30px')
        )

        self.reset_button = widgets.Button(
            description='Reset Selection',
            button_style='warning',
            layout=widgets.Layout(width='150px', height='30px')
        )
        
        # Create buttons
        self.update_label_button = widgets.Button(
            description="Update Labels",
            button_style='success',
            layout=widgets.Layout(width='250px', height='30px')
        )
        self.update_order_button = widgets.Button(
            description="Update Order",
            button_style='success',
            layout=widgets.Layout(width='250px', height='30px')
        )
        self.reset_labelorder_button = widgets.Button(
            description="Reset to Default",
            button_style='warning',
            layout=widgets.Layout(width='250px', height='30px', margin='10px 10px 0 75px')
        )

        # Attach click event handlers
        self.update_label_button.on_click(self.on_update_label_click)
        self.update_order_button.on_click(self.on_update_order_click)
        self.reset_labelorder_button.on_click(self.on_reset_click)

        # Display buttons
        self.label_order_button_box = widgets.HBox([self.update_label_button, self.update_order_button])

        self.var_selection_output = widgets.Output()

        # Set widget events
        self.protein_dropdown.observe(self.update_var_keys, names='value')
        self.search_button.on_click(self.search_var_keys)
        self.add_group_button.on_click(self.add_group)
        self.reset_button.on_click(self.reset_selection)

        
        self.button_box = widgets.HBox([self.search_button, self.add_group_button, self.reset_button],
        layout=widgets.Layout(
            height='40px', 
            width='90%', 
            overflow='hidden', 
            justify_content='space-between'
            )
        )
    # Function to update var_key_dropdown based on selected protein
    def update_var_keys(self, change):
        selected_protein = change['new']
        current_selection = set(self.var_key_dropdown.value)

        new_options = []
        if selected_protein in self.available_grouping_vars:
            new_options = [f"{var}" for var in self.available_grouping_vars[selected_protein]]

        self.var_key_dropdown.options = sorted(set(self.var_key_dropdown.options).union(new_options))
        self.var_key_dropdown.value = tuple(current_selection.intersection(self.var_key_dropdown.options))

    # Function to search and filter var_keys based on the grouping variable text input
    def search_var_keys(self, b):
        group_name = self.grouping_variable_text.value
        if group_name:
            matching_keys = [key for key in self.var_key_dropdown.options if group_name in key]
            self.var_key_dropdown.value = matching_keys
        else:
            with self.var_selection_output:
                self.var_selection_output.clear_output()
                display(HTML('<b style="color:red;">Please enter a group name to search.</b>'))

    # Function to add a group of selected var_keys to the list
    def add_group(self, b):
        selected_protein = self.protein_dropdown.value
        selected_keys = list(self.var_key_dropdown.value)

        if selected_keys and selected_protein:
            combined_keys = [f"{selected_protein}_{key}" for key in selected_keys]
            self.selected_var_keys_list.extend(combined_keys)
            self.selected_var_keys_list = list(set(self.selected_var_keys_list))  # Ensure no duplicates

            with self.var_selection_output:
                self.var_selection_output.clear_output()
                
                
                display(HTML("<h3> </h3>"))

                #display(HTML("<hr style='border: 1px solid black;'>"))
                display(HTML(f"<b>{len(combined_keys)} variables added.</b>"))
                display(HTML(f"<b>Selected variables:</b> {', '.join(combined_keys)}"))
                display(HTML(f"<b>Total unique variables:</b> {len(self.selected_var_keys_list)}"))
                display(HTML(f"<b>All unique variables:</b> {', '.join(self.selected_var_keys_list)}"))

            self.grouping_variable_text.value = ''
            self.filtered_data_variables = self.create_filtered_data_variables()
            # Process data variables
            self.process_data_variables()
            # Create label and order widgets
            self.create_order_input_widgets()
            self.default_label_values = {key: self.label_widgets[key].value for key in self.label_widgets}
            self.default_order_values = [info['label'] for info in self.available_data_variables.values()]
            self.display_label_order_widgets()

        else:
            with self.var_selection_output:
                self.var_selection_output.clear_output()
                display(HTML('<b style="color:red;">Please select a protein and at least one key.</b>'))

    # Function to reset the selection
    def reset_selection(self, b):
        self.selected_var_keys_list.clear()
        self.protein_dropdown.value = None
        self.var_key_dropdown.options = []
        self.grouping_variable_text.value = ''

        self.filtered_data_variables = {}
        self.available_data_variables = {}
        self.label_widgets = {}
        self.order_widgets = {}
        self.default_label_values = {}
        self.default_order_values = []

        with self.var_selection_output:
            self.var_selection_output.clear_output()
            display(HTML('<b style="color:green;">Selection has been reset.</b>'))

        # Clear the label_order_output as well
        self.label_order_output.clear_output()

    # Function to create a filtered dictionary based on selected_var_keys_list
    def create_filtered_data_variables(self):
        return {key: self.data_variables[key] for key in self.selected_var_keys_list if key in self.data_variables}

    # Function to display messages in the output widget
    def display_message(self, message, is_error=False):
        with self.message_output:
            self.message_output.clear_output()  # Clear previous messages
            if is_error:
                display(HTML(f"<b style='color:red;'>{message}</b>"))  # Error message in red
            else:
                display(HTML(f"<b style='color:green;'>{message}</b>"))  # Success message in green

    # Function to update order based on new order input
    def update_order(self, order_labels):

        vars_list = list(self.available_data_variables.keys())
        labels_list = [info['label'] for info in self.available_data_variables.values()]

        # Check if the provided labels match the available labels
        if len(order_labels) != len(labels_list):
            raise ValueError("Number of labels provided does not match the number of items to reorder.")

        # Check for duplicates in order_labels
        if len(order_labels) != len(set(order_labels)):
            raise ValueError("Duplicate labels found in order_labels. Please provide unique labels.")

        # **New Check**: Check for duplicates in available labels
        if len(labels_list) != len(set(labels_list)):
            raise ValueError("Duplicate labels found in available data variables. Cannot reorder unambiguously.")

        # Ensure all provided labels exist in available_data_variables
        if not all(label in labels_list for label in order_labels):
            raise ValueError("One or more provided labels are invalid.")

        # Build a mapping from label to variable key
        label_to_var = {info['label']: var for var, info in self.available_data_variables.items()}

        # Reorder available_data_variables based on the new order of labels
        ordered_available_data_variables = {
            label_to_var[label]: self.available_data_variables[label_to_var[label]] for label in order_labels
        }

        # Update self.available_data_variables
        self.available_data_variables = ordered_available_data_variables

        # Optionally, return the reordered dictionary
        # return self.available_data_variables

    # Event handler for updating labels
    def on_update_label_click(self, b):
        try:
            self.update_labels()
            self.display_message("Labels updated successfully.")
        except Exception as e:
            self.display_message(f"Error updating labels: {e}", is_error=True)

    # Event handler for updating order
    def on_update_order_click(self, b):
        # Split the entered label string and strip spaces
        order_list = [label.strip() for label in self.new_order_input.value.split(',')]
        try:
            # Update order based on the label input
            self.update_order(order_list)
            self.display_message("Order updated successfully.")
        except Exception as e:
            self.display_message(f"Error updating order: {e}", is_error=True)

    # Event handler for resetting labels and order
    def on_reset_click(self, b):
        try:
            # Reset each label widget to its default value
            for key in self.label_widgets:
                self.label_widgets[key].value = self.default_label_values[key]

            # Reset the order widget to its default value
            self.new_order_input.value = ', '.join(self.default_order_values)

            # Apply the default labels and order
            self.on_update_label_click(b)
            self.on_update_order_click(b)

            self.display_message("Labels and order reset to default.")
        except Exception as e:
            self.display_message(f"Error resetting labels and order: {e}", is_error=True)

    # Function to display label and order widgets
    
    def display_label_order_widgets(self):
        # Output widget for displaying messages
        self.message_output = widgets.Output()
    
        # Header for the columns
        header = HTML("<h3><u>Update Sample Labels & Order (Optional)</u></h3>")
    
        # Update labels section
        update_label = [widgets.HTML(value="<h3><u>Update Labels:</u></h3>")]
        for i, (var, info) in enumerate(self.available_data_variables.items()):
            label_widget = HBox([
                widgets.Label(
                    value=f"{i + 1})  {info['label']}  -  {info.get('protein_species', '')}  -  {info.get('protein_name', '')}",
                    layout=widgets.Layout(width='100%', height='30px',overflow='hidden')
                ),
                self.label_widgets.get(var, widgets.Text())
            ])
            update_label.append(label_widget)
        update_label_box = VBox(update_label,  layout=widgets.Layout(margin='0px', height='auto', overflow='visible', padding='0px'))
    
        # Label above the text input box
        label_above_input = widgets.HTML(
            value="<h3><u>Re-order Samples:</u></h3>Enter labels in desired order separated by commas (e.g., label_1, label_2, label_3)")
    
        # Extract labels from available_data_variables for display
        label_list = [info['label'] for info in self.available_data_variables.values()]
    
        # Text input for new order
        # Text input for new order without scrollbar
        self.new_order_input = widgets.Textarea(
            value=', '.join(label_list),
            layout=widgets.Layout(
                width='90%',
                height='auto',  # Automatically adjust the height to fit the content
                overflow='hidden'  # Eliminate scrollbars
            )
        )
        update_order_box = VBox([label_above_input, self.new_order_input], layout=widgets.Layout(margin='0px', height='200px', overflow='visible', padding='0px'))

        # Create buttons with fixed sizes
        update_label_button = widgets.Button(
            description="Update Labels",
            button_style='success',
            layout=widgets.Layout(
                width='250px',       # Fixed width
                height='30px',       # Fixed height
                overflow='hidden'    # Eliminate scrollbars
            )
        )
        
        update_order_button = widgets.Button(
            description="Update Order",
            button_style='success',
            layout=widgets.Layout(
                width='250px',       # Fixed width
                height='30px',       # Fixed height
                overflow='hidden'    # Ensure no internal scrolling
            )
        )
        
        reset_labelorder_button = widgets.Button(
            description="Reset to Default",
            button_style='warning',
            layout=widgets.Layout(
                width='250px',       # Fixed width
                height='30px',       # Fixed height
                overflow='hidden'    # Ensure no internal scrolling
            )
        )
        
        # Combine buttons into a container (HBox) with sufficient width
        label_order_button_box = widgets.HBox(
            [update_label_button, update_order_button, reset_labelorder_button],
            layout=widgets.Layout(
                width='400px',            # Ensure enough space for all buttons
                height='auto',            # Adjust height automatically
                overflow='visible',       # No scrolling for the container
                justify_content='space-between'  # Distribute buttons horizontally
            )
        )

    
        # Attach click event handlers
        update_label_button.on_click(self.on_update_label_click)
        update_order_button.on_click(self.on_update_order_click)
        reset_labelorder_button.on_click(self.on_reset_click)
    
        # Display buttons
        vert_button_box = VBox(
            [
                update_label_box,
                update_order_box,
                label_order_button_box, 
                self.message_output
            ],
            layout=widgets.Layout(
                margin='0px',
                width='90%',        # Ensure it takes up available horizontal space
                height='auto',       # Ensure it takes up as much vertical space as needed
                flex_flow='column',  # Maintain column layout
                align_items='stretch'  # Prevent compacting by stretching items
            )
        )
    
        # Return the constructed widgets
        return vert_button_box#, update_label_box, update_order_box

            
    def update_labels(self):
        # Update labels in available_data_variables based on label_widgets
        for key in self.available_data_variables:
            self.available_data_variables[key]['label'] = self.label_widgets[key].value

    """
    # Function to display the initial selection widgets
    def display_widgets(self):
        display(HTML("<h3><u>Select Protein and Grouping Variables:</u></h3>"))
        display(self.protein_dropdown)
        display(self.grouping_variable_text)
        display(self.var_key_dropdown)
        display(self.button_box)
        display(self.var_selection_output)
        display(self.label_order_output)
        #display(self.vert_button_box)
    """;    
    # Function to display the initial selection widgets
            
    def display_widgets(self):
        # Create a grid layout with 3 rows and 2 columns
    
        # Input widgets
        input_widgets = VBox([
            HTML("<h3><u>Select Protein and Grouping Variables:</u></h3>"),
            self.protein_dropdown,
            self.grouping_variable_text,
            self.var_key_dropdown,
            self.button_box,
            self.label_order_output
        ], layout=widgets.Layout(height = 'auto', margin='0px', padding='0px',overflow='hidden',))  # Minimize widget margins

    
        # Output widgets
        output_widgets = VBox([
            self.var_selection_output,
        ], layout=widgets.Layout(margin='0px', padding='0px'))  # Minimize widget margins

        vert_button_box = self.display_label_order_widgets()

    
        # Display the grid
        #display(grid)
        return input_widgets, output_widgets, vert_button_box#, update_label_box, update_order_box

In [17]:

class HeatmapPlotHandler:
    def __init__(self, selector):
        instance_variables = {
            attr: getattr(selector, attr)
            for attr in dir(selector)
            if not callable(getattr(selector, attr))  # Exclude methods
            and not attr.startswith("__")            # Exclude magic methods
            and "button" not in attr                 # Exclude attributes containing "button"
        }
        for key, value in instance_variables.items():
            setattr(self, key, value)
  
        # Initialize with data from selector
        self.plot_heatmap, self.plot_zero = 'yes', 'no'

        # List of valid gradient colormaps
        def get_valid_gradient_colormaps():
            return settings.valid_gradient_cmaps
        
        # List of valid discrete colormaps
        def get_valid_discretecolormaps():
            return settings.valid_discrete_cmaps

        def display_plotting_options(self):
            dropdown_layout = widgets.Layout(width='400px')
            self.plot_message = widgets.HTML("<h3><u>Ploting Options:</u></h3>")

            self.ms_average_choice_dropdown = widgets.Dropdown(
                options=['yes', 'no'],
                description='Plot Averaged Data:',
                disabled=False,
                style={'description_width': 'initial'},
                layout=dropdown_layout,
            )
            self.bio_or_pep_dropdown = widgets.Dropdown(
                options=[('None', 'no'), ('Peptide Intervals', '1'), ('Bioactive Functions', '2')],
                description='Plot Specific Peptides:',
                disabled=False,
                style={'description_width': 'initial'},
                layout=dropdown_layout,
            )
            self.specific_select_multiple = widgets.SelectMultiple(
                options=[],
                description='Specific Options:',
                disabled=False,
                layout=widgets.Layout(display='none')  # Start hidden
            )
        
            # Attach the observer only to bio_or_pep_dropdown
            self.bio_or_pep_dropdown.observe(
                lambda change: on_selection_change(self, change),
                names='value'
            )

        
        def create_plotting_widgets(self):
            # Generate filenames
            generate_filenames(self)
    
            # Layouts
            description_layout_invisible = widgets.Layout(width='400px', overflow = 'visible')
            description_layout = widgets.Layout(width='400px', overflow = 'visible')
            dropdown_layout = widgets.Layout(width='400px', overflow = 'visible')
            dropdown_layout_large = widgets.Layout(width='400px', overflow = 'visible')
    
            # Color Widgets
            self.hm_selected_color = widgets.Dropdown(
                options=get_valid_gradient_colormaps(),
                value=default_hm_color,
                description='Heatmap:',
                layout=dropdown_layout,
                style={'description_width': 'initial'}
            )
    
            self.lp_selected_color = widgets.Dropdown(
                options=get_valid_discretecolormaps(),
                value=default_lp_color,
                description='Line Plot:',
                layout=dropdown_layout,
                style={'description_width': 'initial'}
            )
    
            self.avglp_selected_color = widgets.Dropdown(
                options=valid_discrete_cmaps,
                value=default_avglp_color,
                description='Avg Line Plot:',
                layout=dropdown_layout,
                style={'description_width': 'initial'}
            )
    
            self.color_message = widgets.HTML("<h3><u>Color Options:</u></h3>")
            self.color_widget_box = widgets.VBox([
                self.color_message,
                self.hm_selected_color,
                self.lp_selected_color,
                self.avglp_selected_color
            ])
    
            # Figure Label Widgets
            self.xaxis_label_input = widgets.Text(
                value=f"{self.protein_name_short} Sequence",
                description='x-axis label:',
                layout=description_layout,
                style={'description_width': 'initial'}
            )
    
            self.yaxis_label_input = widgets.Text(
                value="Averaged Peptide Abundance",
                description='y-axis label:',
                layout=description_layout,
                style={'description_width': 'initial'}
            )
    
            self.yaxis_position = widgets.IntSlider(
                value=0,
                min=-10,
                max=10,
                step=1,
                layout=description_layout,
                description='y-axis title position:',
                style={'description_width': 'initial'}
            )
    
            
            self.legend_title_input_1 = widgets.Text(
                value=legend_title[0],
                description=f'Legend title ({legend_title[0]}):',
                layout=description_layout,
                style={'description_width': 'initial'}

            )
            
            self.legend_title_input_2 = widgets.Text(
                value=legend_title[1],
                description=f'Legend title ({legend_title[1]}):',
                layout=description_layout,
                style={'description_width': 'initial'}

            )
            
            self.legend_title_input_3 = widgets.Text(
                value=legend_title[2],
                description=f'Legend title ({legend_title[2]}):',
                layout=description_layout,
                style={'description_width': 'initial'}

            )
            
            self.legend_title_input_4 = widgets.Text(
                value=legend_title[3],
                description=f'Legend title ({legend_title[3]}):',
                layout=description_layout,
                style={'description_width': 'initial'}

            )
            
            self.legend_title_input_5 = widgets.Text(
                value=legend_title[4],
                description=f'Legend title ({legend_title[4]}):',
                layout=description_layout,
                style={'description_width': 'initial'}
            )
            # Conditional Widgets
            if self.ms_average_choice == 'yes' and self.bio_or_pep == '1':
                self.legend_title_input_1 = widgets.Text(
                    value=self.legend_title[0],
                    description=f'Legend title ({self.legend_title[0]}):',
                    layout=description_layout,
                    style={'description_width': 'initial'}
                )
                self.legend_title_input_3 = widgets.Text(
                    value=self.legend_title[2],
                    description=f'Legend title ({self.legend_title[2]}):',
                    layout=description_layout_invisible,
                    style={'description_width': 'initial'}
                )
                self.legend_title_input_3.layout.display = 'none'
                self.legend_title_input_4 = widgets.Text(
                    value=self.legend_title[3],
                    description=f'Legend title ({self.legend_title[3]}):',
                    layout=description_layout,
                    style={'description_width': 'initial'}
                )
                self.legend_title_input_5 = widgets.Text(
                    value=self.legend_title[4],
                    description=f'Legend title ({self.legend_title[4]}):',
                    layout=description_layout,
                    style={'description_width': 'initial'}
                )

            if self.ms_average_choice == 'yes' and self.bio_or_pep == '2':
                self.legend_title_input_1 = widgets.Text(
                    value=self.legend_title[0],
                    description=f'Legend title ({self.legend_title[0]}):',
                    layout=description_layout,
                    style={'description_width': 'initial'}
                )
                self.legend_title_input_3 = widgets.Text(
                    value=self.legend_title[2],
                    description=f'Legend title ({self.legend_title[2]}):',
                    layout=description_layout,
                    style={'description_width': 'initial'}
                )
                self.legend_title_input_4 = widgets.Text(
                    value=self.legend_title[3],
                    description=f'Legend title ({self.legend_title[3]}):',
                    layout=description_layout_invisible,
                    style={'description_width': 'initial'}
                )
                self.legend_title_input_4.layout.display = 'none'
                self.legend_title_input_5 = widgets.Text(
                    value=self.legend_title[4],
                    description=f'Legend title ({self.legend_title[4]}):',
                    layout=description_layout,
                    style={'description_width': 'initial'}
                )
            
            if self.ms_average_choice == 'yes' and self.bio_or_pep == 'no':
                self.legend_title_input_1 = widgets.Text(
                    value=self.legend_title[0],
                    description=f'Legend title ({self.legend_title[0]}):',
                    layout=description_layout,
                    style={'description_width': 'initial'}
                )
                self.legend_title_input_3 = widgets.Text(
                    value=self.legend_title[2],
                    description=f'Legend title ({self.legend_title[2]}):',
                    layout=description_layout_invisible,
                    style={'description_width': 'initial'}
                )
                self.legend_title_input_3.layout.display = 'none'
                self.legend_title_input_4 = widgets.Text(
                    value=self.legend_title[3],
                    description=f'Legend title ({self.legend_title[3]}):',
                    layout=description_layout_invisible,
                    style={'description_width': 'initial'}
                )
                self.legend_title_input_4.layout.display = 'none'
                self.legend_title_input_5 = widgets.Text(
                    value=self.legend_title[4],
                    description=f'Legend title ({self.legend_title[4]}):',
                    layout=description_layout,
                    style={'description_width': 'initial'}
                )
            
            if self.ms_average_choice == 'no' and self.bio_or_pep == '1':
                self.legend_title_input_1 = widgets.Text(
                    value=self.legend_title[0],
                    description=f'Legend title ({self.legend_title[0]}):',
                    layout=description_layout,
                    style={'description_width': 'initial'}
                )
                self.legend_title_input_3 = widgets.Text(
                    value=self.legend_title[2],
                    description=f'Legend title ({self.legend_title[2]}):',
                    layout=description_layout_invisible,
                    style={'description_width': 'initial'}
                )
                self.legend_title_input_3.layout.display = 'none'
                self.legend_title_input_4 = widgets.Text(
                    value=self.legend_title[3],
                    description=f'Legend title ({self.legend_title[3]}):',
                    layout=description_layout,
                    style={'description_width': 'initial'}
                )
                self.legend_title_input_5 = widgets.Text(
                    value=self.legend_title[4],
                    description=f'Legend title ({self.legend_title[4]}):',
                    layout=description_layout_invisible,
                    style={'description_width': 'initial'}
                )
                self.legend_title_input_5.layout.display = 'none'
            
            if self.ms_average_choice == 'no' and self.bio_or_pep == '2':
                self.legend_title_input_1 = widgets.Text(
                    value=self.legend_title[0],
                    description=f'Legend title ({self.legend_title[0]}):',
                    layout=description_layout,
                    style={'description_width': 'initial'}
                )
                self.legend_title_input_3 = widgets.Text(
                    value=self.legend_title[2],
                    description=f'Legend title ({self.legend_title[2]}):',
                    layout=description_layout,
                    style={'description_width': 'initial'}
                )
                self.legend_title_input_4 = widgets.Text(
                    value=self.legend_title[3],
                    description=f'Legend title ({self.legend_title[3]}):',
                    layout=description_layout_invisible,
                    style={'description_width': 'initial'}
                )
                self.legend_title_input_4.layout.display = 'none'
                self.legend_title_input_5 = widgets.Text(
                    value=self.legend_title[4],
                    description=f'Legend title ({self.legend_title[4]}):',
                    layout=description_layout_invisible,
                    style={'description_width': 'initial'}
                )
                self.legend_title_input_5.layout.display = 'none'
            
            # Plot Widgets
            self.plot_port = widgets.ToggleButton(
                value=True,
                description='Portrait Plot',
                disabled=False,
                button_style='',
                tooltip='Show updated plot',
                icon='check'
            )
    
            self.plot_land = widgets.ToggleButton(
                value=True,
                description='Landscape Plot',
                disabled=False,
                button_style='',
                tooltip='Show updated plot',
                icon='check'
            )
                 
            self.create_plot_message = widgets.HTML("<h3><u>Create Plot Checkboxs:</u></h3>")
           
            self.plot_toggle_buttons = widgets.HBox([
                self.plot_port,
                self.plot_land
            ])
            
            self.plot_toggle_widget_box = widgets.VBox([
                self.create_plot_message,
                self.plot_toggle_buttons,

            ])     
            self.figure_label_message = widgets.HTML("<h3><u>Figure Label Options:</u></h3>")
            
            self.figure_label_box = widgets.VBox([
                self.figure_label_message,
                self.xaxis_label_input,
                self.yaxis_label_input,
                self.yaxis_position,
                self.legend_title_input_1,
                self.legend_title_input_2,
                self.legend_title_input_3,
                self.legend_title_input_4,
                self.legend_title_input_5,
                self.plot_toggle_widget_box 
            ], layout=widgets.Layout(
            width='500px',
            height='400px',
            margin='0px')
            )


      
            self.filename_port_input = widgets.Text(
                value=self.display_filename_port,
                description='Filename (Portrait):',
                layout=dropdown_layout_large,
                style={'description_width': 'initial'}
            )
    
            self.filename_land_input = widgets.Text(
                value=self.display_filename_land,
                description='Filename (Landscape):',
                layout=dropdown_layout_large,
                style={'description_width': 'initial'}
            )
            self.filename_label_message = widgets.HTML("<h3><u>Save As Options</u></h3>")

            self.plot_filename_widget_box = widgets.VBox([
                self.filename_label_message,
                self.filename_port_input,
                self.filename_land_input
            ])

            # Add buttons for update and save plot
            self.update_button = widgets.Button(
                description='Show/Update Plot',
                button_style='success',
                tooltip='Click to update the plot',
                icon='refresh'
            )
    
            self.save_button = widgets.Button(
                description='Save Plot',
                button_style='info',
                tooltip='Click to save the plot',
                icon='save'
            )

            self.update_save_box = widgets.HBox([self.update_button, self.save_button])

        def on_dropdown_change(self, change):
            self.ms_average_choice = self.ms_average_choice_dropdown.value
            self.bio_or_pep = self.bio_or_pep_dropdown.value
            if self.bio_or_pep != 'no':
                self.selected_bio_or_pep = self.specific_select_multiple.value
            else:
                self.selected_bio_or_pep = []
    
            if self.bio_or_pep != 'no' and self.selected_bio_or_pep:
                self.selected_peptides, self.selected_functions = proceed_with_label_specific_options(self.selected_bio_or_pep, self.bio_or_pep)
            else:
                self.selected_peptides, self.selected_functions = [], []
    
            # Call the method to create plotting widgets
            create_plotting_widgets(self)

        
        # Function to handle updates

        def extract_non_zero_non_nan_values(df):
            unique_functions = set()
            # Iterate over each value in the DataFrame
            for value in df.stack().values:  # df.stack() stacks the DataFrame into a Series
                if value != 0 and not pd.isna(value):  # Check if value is non-zero and not NaN
                    if isinstance(value, str):
                        # If the value is a string, it could contain multiple delimited entries
                        entries = value.split('; ')
                        unique_functions.update(entries)
                    else:
                        unique_functions.add(value)
            return unique_functions
    

        def on_selection_change(self, change):
            if change['type'] == 'change' and change['name'] == 'value':
                self.bio_or_pep = self.bio_or_pep_dropdown.value
    
                # Initialize containers for unique values
                unique_functions = set()
                unique_peptides = set()
    
                # Aggregate unique functions and peptides from available data
                for var in self.available_data_variables:
                    df = self.available_data_variables[var]['bioactive_peptide_func_df']
                    df.replace('0', 0, inplace=True)  # Standardize zero representations
                    unique_functions.update(extract_non_zero_non_nan_values(df))
    
                    abs_df = self.available_data_variables[var]['bioactive_peptide_abs_df']
                    unique_peptides.update(col for col in abs_df.columns if col not in ['AA', 'count', 'average'])
    
                # Convert sets to sorted lists for widget options
                unique_peptides_list = sorted(list(unique_peptides))
                unique_functions_list = sorted(list(unique_functions))
    
                # Update widget based on dropdown choice
                if self.bio_or_pep == '1':  # Peptide Intervals
                    self.specific_select_multiple.options = [(peptide, peptide) for peptide in unique_peptides_list]
                    self.specific_select_multiple.layout.display = 'block'
                elif self.bio_or_pep == '2':  # Bioactive Functions
                    self.specific_select_multiple.options = [(function, function) for function in unique_functions_list]
                    self.specific_select_multiple.layout.display = 'block'
                else:
                    self.specific_select_multiple.options = [""]
                    self.specific_select_multiple.layout.display = 'none'
        # Function to display plotting options    
        def generate_additional_vars_str(self):
            additional_vars = []
    
            if self.plot_heatmap == 'yes':
                additional_vars.append('heatmap')
            elif self.plot_heatmap == 'no':
                additional_vars.append('no-heatmap')
    
            if self.bio_or_pep == '1':
                additional_vars.append('intervals')
            elif self.bio_or_pep == '2':
                additional_vars.append('bioactive-functions')
            elif self.bio_or_pep == 'no':
                additional_vars.append('averages-only')
    
            # Join the additional_vars list into a single string with underscores
            additional_vars_str = '_'.join(additional_vars)
            return additional_vars_str
    
        def generate_filenames(self):
            additional_vars_str = generate_additional_vars_str(self)
            self.xaxis_label = f"\n{self.protein_name_short.replace('_', ' ')} Sequence"
            self.yaxis_label = 'Averaged Peptide Abundance'
    
            self.protein_filename_short = re.sub(r'[^\w-]', '-', self.protein_name_short)
            self.display_filename_port = f'portrait_{self.user_protein_id}_{self.protein_filename_short}_average-only'
            self.display_filename_land = f'landscape_{self.user_protein_id}_{self.protein_filename_short}_{additional_vars_str}'
    
        # Call the method to create plotting widgets
        create_plotting_widgets(self)

        # Attach observer functions to widgets
        display_plotting_options(self)
        # Attach observer functions to widgets
        self.ms_average_choice_dropdown.observe(
            lambda change: on_dropdown_change(self, change), names='value'
        )
        self.bio_or_pep_dropdown.observe(
            lambda change: on_dropdown_change(self, change), names='value'
        )
        self.specific_select_multiple.observe(
            lambda change: on_dropdown_change(self, change), names='value'
        )

        # Manually trigger the function once to use default values at the start
           #on_dropdown_change(self, None)
 

    
        # Create plot output widget
        self.plot_output = widgets.Output(layout=widgets.Layout(
        width='100%',
        height='100%',  # Automatically adjust the height to fit the content
        #overflow='hidden'  # Eliminate scrollbars)
        ))
                # Attach button click events
        self.update_button.on_click(self.on_update_plot_clicked)
        self.save_button.on_click(self.on_save_plot_clicked)
             
    #self.plot_output.capture(clear_output=True)
    def on_update_plot_clicked(self, b):
        with self.plot_output:
            # Clear the previous output
            self.plot_output.clear_output()
    
            # Call the update_plot function
            figures = update_plot(
                self.available_data_variables, self.ms_average_choice, self.bio_or_pep, self.selected_peptides, 
                self.selected_functions, self.hm_selected_color.value, self.lp_selected_color.value, 
                self.avglp_selected_color.value, self.xaxis_label_input.value, self.yaxis_label_input.value, 
                self.yaxis_position.value, self.legend_title_input_1.value, self.legend_title_input_2.value, 
                self.legend_title_input_3.value, self.legend_title_input_4.value, self.legend_title_input_5.value, 
                self.plot_land.value, self.plot_port.value, self.filename_port_input.value, 
                self.filename_land_input.value, save_fig='no'
            )
            return figures
    
    #self.plot_output.capture(clear_output=True)
    def on_save_plot_clicked(self, b):
        with self.plot_output:
            # Clear the previous output
            self.plot_output.clear_output()
    
            # Update filenames based on user input
            self.filename_land, self.filename_port = update_filenames(
                self.filename_port_input.value, self.filename_land_input.value
            )
    
            # Call the update_plot function with save option enabled
            figures = update_plot(
                self.available_data_variables, self.ms_average_choice, self.bio_or_pep, self.selected_peptides, 
                self.selected_functions, self.hm_selected_color.value, self.lp_selected_color.value, 
                self.avglp_selected_color.value, self.xaxis_label_input.value, self.yaxis_label_input.value, 
                self.yaxis_position.value, self.legend_title_input_1.value, self.legend_title_input_2.value, 
                self.legend_title_input_3.value, self.legend_title_input_4.value, self.legend_title_input_5.value, 
                self.plot_land.value, self.plot_port.value, self.filename_port, self.filename_land, save_fig='yes'
            )
    
            # Notify the user that files have been saved
            display(HTML(f'Files have been saved to the <b>heatmap_images</b> directory'))
    """            
    def get_layout(self):
        # Display the widgets as needed
        display(HTML(f"<h3><u>Select line plot options</u></h3>"))
        display(self.ms_average_choice_dropdown)
        display(self.bio_or_pep_dropdown)
        display(self.specific_select_multiple)
        display(self.color_widget_box)
        display(self.figure_label_box)
        display(self.plot_filename_widget_box)
        display(self.update_save_box)
        display(self.plot_output)
        return  # Suppress implicit None
    """;
    def get_layout(self):
        # Create a grid layout with 4 rows and 3 columns
        grid = GridspecLayout(
            1, 2,  # Number of rows and columns
            width='1200px', 
            height='auto',
            grid_gap='5px',  # Adjust spacing between grid elements
        )
            
        # Row 0, Column 0: Input widgets
        input_widgets = VBox([
            self.plot_message,
            self.ms_average_choice_dropdown,
            self.bio_or_pep_dropdown,
            self.specific_select_multiple,
            self.color_widget_box,
            self.figure_label_box,
            self.plot_filename_widget_box,
            self.update_save_box
        ], layout=widgets.Layout(
            width='90%',
            margin='10px'
        ))
        grid[0, 0] = input_widgets  # Place in row 0, column 0
        
        return grid
    
        #return input_widgets,  self.figure_label_box, self.update_save_box

    def show_plots(self):
        display(self.plot_output)  # Span across all columns in row 3
    
        return 

In [28]:

# Initialize the selector
selector = HeatmapDataHandler()

# Create an output container for dynamic updates
dynamic_output = widgets.Output()

# Placeholder for the app
app = None

def generate_grid(selector):
    global app  # Ensure global app instance is updated

    # Reinitialize the app with updated data
    app = HeatmapPlotHandler(selector)

    # Get widgets from the selector
    sel_input_widgets, sel_output_widgets, sel_vert_button_box = selector.display_widgets()

    # Set fixed heights for widgets
    sel_input_widgets.layout.height = '300px'
    sel_output_widgets.layout.height = '300px'
    sel_vert_button_box.layout.height = '300px'

    # Create a grid layout with fixed row heights
    grid = GridspecLayout(
        2, 2,  # Number of rows and columns
        width='1200px', 
        grid_gap='5px',  # Adjust spacing between grid elements
    )

    # Add widgets to the grid
    grid[0, 0] = sel_input_widgets  # Place in row 0, column 0
    grid[0, 1] = sel_output_widgets  # Place in row 0, column 1
    grid[1, 0] = sel_vert_button_box  # Place in row 1, column 0

    # Return the grid layout
    return grid

# Function to dynamically update and reinitialize the app
def show_plotting_options(change=None):
    global app  # Use the global app instance

    with dynamic_output:
        # Clear the previous content
        clear_output(wait=True)

        # Generate and display the grid with updated widgets
        grid = generate_grid(selector)
        display(grid)

        # Display additional widgets from the app layout
        app.get_layout()  # This will directly display self.input_widgets
        display(app.get_layout())

        # Display the plots (if any)
        plots = app.show_plots()
        display(plots)

# Attach observers to relevant widgets in HeatmapDataHandler
selector.add_group_button.on_click(show_plotting_options)
selector.reset_button.on_click(show_plotting_options)
selector.update_label_button.on_click(show_plotting_options)
selector.update_order_button.on_click(show_plotting_options)
selector.reset_labelorder_button.on_click(show_plotting_options)

# Initial display
with dynamic_output:
    grid = generate_grid(selector)
    display(grid)
    display(selector.label_order_output)
    if app:
        display(app.get_layout())
        display(app.show_plots())


# Display the dynamic output container
display(dynamic_output)

Output()